# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_map_csv = "../WeatherPy/City_Weather.csv"
weather_map_df = pd.DataFrame(pd.read_csv(weather_map_csv))
weather_map_df.head()

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Carnarvon,-24.8667,113.6333,AU,1613188378,87.80,74,90,5.75
1,Palekastro,35.2000,26.2500,GR,1613188380,55.00,70,33,1.99
2,Kahului,20.8947,-156.4700,US,1613188380,80.60,54,40,21.85
3,Dikson,73.5069,80.5464,RU,1613188381,-25.98,87,98,11.39
4,Thompson,55.7435,-97.8558,CA,1613188382,-34.60,74,1,1.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Use API gkey to access info about maps 
gmaps.configure(api_key = g_key)

In [6]:
#Identify locations as lat and lng/humidity as weight -- convert to float to fill NaN values
locations = weather_map_df[["Latitude","Longitude"]]
humidity = weather_map_df["Humidity"].astype(float)

#Plot Google Heatmap
figure = gmaps.figure(zoom_level = 1.80, center = (20, 5))

#Set max intensity based on highest humidity in the df
max_intensity = weather_map_df["Humidity"].max()

#create and combine heat layer on top of Google map
heat_map_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_intensity, point_radius=3)

figure.add_layer(heat_map_layer)

figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
